In [65]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from torch.utils.data import DataLoader, TensorDataset
import joblib
#!pip install onnx
import onnx
import torch.onnx

In [23]:
#!pip list

In [66]:
df = pd.read_csv('merged.csv')  # Ersetzen Sie 'data.csv' durch den Pfad Ihrer Datei

# Features und Labels extrahieren
features = df[['x', 'y', 'z', 'yaw', 'pitch', 'roll']].values
labels = df['action'].values

# Labels encodieren
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

# Feature-Skalierung
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)

# Daten in Trainings- und Testdaten aufteilen
X_train, X_test, y_train, y_test = train_test_split(scaled_features, encoded_labels, test_size=0.2, random_state=42)

# Daten in Tensoren umwandeln
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# DataLoader erstellen
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [67]:
# LSTM-Modell definieren
class LSTMClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers):
        super(LSTMClassifier, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x, h0=None, c0=None):
        if h0 is None or c0 is None:
          h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
          c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        out, _ = self.lstm(x, (h0, c0))
        out = out[:, -1, :]
        out = self.fc(out)
        return out

In [68]:
# Hyperparameter
input_size = 6  # Anzahl der Features
hidden_size = 128
output_size = len(label_encoder.classes_)
num_layers = 2
num_epochs = 500
learning_rate = 0.001

# Modell, Verlustfunktion und Optimierer instanziieren
model = LSTMClassifier(input_size, hidden_size, output_size, num_layers)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [69]:
# Training des Modells
model.train()
for epoch in range(num_epochs):
    for X_batch, y_batch in train_loader:
        X_batch = X_batch.unsqueeze(1)  # Batch-Dimension hinzufügen für LSTM
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [1/500], Loss: 0.1641
Epoch [2/500], Loss: 0.0344
Epoch [3/500], Loss: 0.0958
Epoch [4/500], Loss: 0.0073
Epoch [5/500], Loss: 0.0399
Epoch [6/500], Loss: 0.0158
Epoch [7/500], Loss: 0.1001
Epoch [8/500], Loss: 0.1346
Epoch [9/500], Loss: 0.0173
Epoch [10/500], Loss: 0.2591
Epoch [11/500], Loss: 0.0400
Epoch [12/500], Loss: 0.0242
Epoch [13/500], Loss: 0.0051
Epoch [14/500], Loss: 0.0015
Epoch [15/500], Loss: 0.0048
Epoch [16/500], Loss: 0.0017
Epoch [17/500], Loss: 0.1204
Epoch [18/500], Loss: 0.0245
Epoch [19/500], Loss: 0.0026
Epoch [20/500], Loss: 0.0043
Epoch [21/500], Loss: 0.0015
Epoch [22/500], Loss: 0.0049
Epoch [23/500], Loss: 0.0403
Epoch [24/500], Loss: 0.0010
Epoch [25/500], Loss: 0.0004
Epoch [26/500], Loss: 0.0008
Epoch [27/500], Loss: 0.0328
Epoch [28/500], Loss: 0.0026
Epoch [29/500], Loss: 0.0020
Epoch [30/500], Loss: 0.0046
Epoch [31/500], Loss: 0.2657
Epoch [32/500], Loss: 0.0019
Epoch [33/500], Loss: 0.0006
Epoch [34/500], Loss: 0.0008
Epoch [35/500], Loss: 0

In [70]:
# Save the trained model
torch.save(model.state_dict(), 'gesture_model.pth')
joblib.dump(scaler, 'scaler.pkl')
joblib.dump(label_encoder, 'label_encoder.pkl')

['label_encoder.pkl']

In [71]:
# Modell bewerten
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for X_batch, y_batch in test_loader:
        X_batch = X_batch.unsqueeze(1)
        outputs = model(X_batch)
        _, predicted = torch.max(outputs.data, 1)
        total += y_batch.size(0)
        correct += (predicted == y_batch).sum().item()

    accuracy = correct / total
    print(f'Accuracy: {accuracy * 100:.2f}%')

Accuracy: 99.33%


In [72]:
# Testen mit neuen Daten
def predict_new_data(new_data, model, scaler, label_encoder):
    # Neue Daten skalieren
    new_data_scaled = scaler.transform(new_data)

    # In Tensor umwandeln
    new_data_tensor = torch.tensor(new_data_scaled, dtype=torch.float32).unsqueeze(1)  # Batch-Dimension hinzufügen

    # Modell in den Evaluierungsmodus setzen
    model.eval()

    with torch.no_grad():
        outputs = model(new_data_tensor)
        _, predicted = torch.max(outputs.data, 1)

    # Vorhersagen decodieren
    predicted_labels = label_encoder.inverse_transform(predicted.cpu().numpy())

    return predicted_labels

In [73]:
# Testen mit anderen Daten
"""
-5019.0,23470.0,7634.0,-14.360418,-19.668957,-19.876141,Neutral
-5591.0,27582.0,7211.0,-13.529002,-18.567438,-19.57518,Neutral
-8369.0,26687.0,7139.0,-13.811018,-18.471643,-18.311039,Neutral
-11699.0,22100.0,7059.0,-15.035475,-19.277781,-14.932572,Neutral
-15578.0,17427.0,5996.0,-21.207737,-20.4666,-12.251103,Neutral
-25273.0,10737.0,2789.0,-29.427649,-23.855116,-8.140183,Crouch
-24939.0,4538.0,4197.0,-47.376305,-26.376276,-4.485165,Crouch
-25005.0,12752.0,5266.0,-62.027802,-28.546284,2.15706,Crouch
-26196.0,-15782.0,6594.0,-88.199348,-28.281425,6.477275,Crouch
-27253.0,-22324.0,914.0,-102.161095,-23.648937,16.214033,Crouch
-27887.0,-19877.0,4589.0,-113.087097,-19.671341,22.485985,Crouch
-27773.0,-1554.0,10906.0,-117.005463,-17.334806,22.236414,Crouch
-26835.0,-6510.0,6585.0,-121.647881,-17.750044,16.642509,Crouch
-26077.0,-10775.0,3774.0,-122.92939,-20.976959,7.598702,Crouch
-25294.0,-17870.0,2209.0,-122.441956,-24.102129,7.248987,Crouch
-24993.0,-23676.0,-860.0,-120.279739,-27.141514,10.803078,Crouch
-24331.0,-20957.0,-1861.0,-113.666977,-28.32806,13.204218,Crouch
-23710.0,-15666.0,-3742.0,-107.232986,-30.993572,12.760225,Crouch
-22933.0,-13027.0,-4191.0,-95.226784,-33.565964,9.099196,Crouch
-22730.0,-5367.0,-2946.0,-86.321915,-36.888329,1.072562,Crouch
-22941.0,2860.0,-1892.0,-72.129295,-37.782143,-4.732635,Crouch
-23380.0,11539.0,962.0,-62.240501,-36.857086,-13.824538,Crouch
-20222.0,19113.0,3676.0,-49.183418,-34.959476,-19.523731,Neutral
-15408.0,23036.0,7302.0,-42.288437,-31.399384,-25.70689,Neutral
-13664.0,24529.0,9320.0,-33.258495,-29.620506,-28.231247,Neutral
-9466.0,24071.0,10478.0,-27.782314,-27.430836,-29.805155,Neutral
-7136.0,23920.0,10572.0,-19.93651,-26.271467,-29.415785,Neutral
"""

new_data1 = [
    [-5019.0,23470.0,7634.0,-14.360418,-19.668957,-19.876141],
    [-5591.0,27582.0,7211.0,-13.529002,-18.567438,-19.57518],
    [-8369.0,26687.0,7139.0,-13.811018,-18.471643,-18.311039],
    [-11699.0,22100.0,7059.0,-15.035475,-19.277781,-14.932572],
    [-15578.0,17427.0,5996.0,-21.207737,-20.4666,-12.251103],
    [-25273.0,10737.0,2789.0,-29.427649,-23.855116,-8.140183],
    [-24939.0,4538.0,4197.0,-47.376305,-26.376276,-4.485165],
    [-25005.0,12752.0,5266.0,-62.027802,-28.546284,2.15706],
    [-26196.0,-15782.0,6594.0,-88.199348,-28.281425,6.477275],
    [-27253.0,-22324.0,914.0,-102.161095,-23.648937,16.214033],
    [-27887.0,-19877.0,4589.0,-113.087097,-19.671341,22.485985],
    [-27773.0,-1554.0,10906.0,-117.005463,-17.334806,22.236414],
    [-26835.0,-6510.0,6585.0,-121.647881,-17.750044,16.642509],
    [-26077.0,-10775.0,3774.0,-122.92939,-20.976959,7.598702],
    [-25294.0,-17870.0,2209.0,-122.441956,-24.102129,7.248987],
    [-24993.0,-23676.0,-860.0,-120.279739,-27.141514,10.803078],
    [-24331.0,-20957.0,-1861.0,-113.666977,-28.32806,13.204218],
    [-23710.0,-15666.0,-3742.0,-107.232986,-30.993572,12.760225],
    [-22933.0,-13027.0,-4191.0,-95.226784,-33.565964,9.099196],
    [-22730.0,-5367.0,-2946.0,-86.321915,-36.888329,1.072562],
    [-22941.0,2860.0,-1892.0,-72.129295,-37.782143,-4.732635],
    [-23380.0,11539.0,962.0,-62.240501,-36.857086,-13.824538],
    [-20222.0,19113.0,3676.0,-49.183418,-34.959476,-19.523731],
    [-15408.0,23036.0,7302.0,-42.288437,-31.399384,-25.70689],
    [-13664.0,24529.0,9320.0,-33.258495,-29.620506,-28.231247],
    [-9466.0,24071.0,10478.0,-27.782314,-27.430836,-29.805155],
    [-7136.0,23920.0,10572.0,-19.93651,-26.271467,-29.415785]
] # Ersetzen Sie dies durch Ihre neuen Daten


"""
21710.0,3934.0,940.0,-4.352364,-13.844193,-12.584866,Neutral
11601.0,7937.0,1029.0,-4.594508,-11.435,-13.1503,Neutral
5178.0,5437.0,910.0,-5.920663,-9.351481,-13.070343,Neutral
-8941.0,4831.0,813.0,-8.764639,-9.310511,-13.256666,Neutral
-16675.0,9424.0,1067.0,-11.221966,-11.122966,-17.239277,Neutral
-28832.0,20976.0,1246.0,-17.419025,-13.899675,-23.136898,Neutral
-27144.0,16766.0,-952.0,-22.814985,-20.076349,-33.424778,Shoot
-25726.0,-9456.0,-836.0,-31.062357,-25.452982,-36.934464,Shoot
-23457.0,-6729.0,8890.0,-36.388248,-34.632843,-41.479675,Shoot
-13342.0,5495.0,15403.0,-38.879993,-39.638306,-46.69796,Shoot
-13950.0,4022.0,11844.0,-36.205753,-42.41032,-48.735119,Shoot
-17379.0,1175.0,3612.0,-29.3762,-40.888321,-43.87096,Shoot
-17644.0,5237.0,906.0,-25.636518,-35.147442,-36.420631,Shoot
-13387.0,14061.0,1121.0,-20.844109,-30.878424,-35.420471,Shoot
-5536.0,17643.0,2526.0,-17.503853,-25.282803,-37.302792,Shoot
-441.0,18073.0,5692.0,-12.796161,-22.123541,-37.482185,Shoot
-1052.0,13925.0,5458.0,-10.660426,-17.78968,-33.799526,Shoot
1416.0,3755.0,3915.0,-8.636809,-15.191227,-28.779388,Shoot
4216.0,704.0,-45.0,-7.520261,-12.325178,-21.302486,Neutral
3867.0,7068.0,706.0,-5.716715,-10.736494,-18.8957,Neutral
3917.0,4603.0,-865.0,-4.368924,-7.545374,-17.54945,Neutral
4891.0,163.0,-3398.0,-2.847019,-5.442245,-15.319879,Neutral
2282.0,-2054.0,-2474.0,-2.491301,-3.002312,-10.408621,Neutral
2751.0,-7751.0,843.0,-4.044754,-2.411417,-7.670542,Neutral
-898.0,-12807.0,2671.0,-5.888142,-3.370853,-4.061652,Neutral
"""

new_data2 = [
    [21710.0, 3934.0, 940.0, -4.352364, -13.844193, -12.584866],
    [11601.0, 7937.0, 1029.0, -4.594508, -11.435, -13.1503],
    [5178.0, 5437.0, 910.0, -5.920663, -9.351481, -13.070343],
    [-8941.0, 4831.0, 813.0, -8.764639, -9.310511, -13.256666],
    [-16675.0, 9424.0, 1067.0, -11.221966, -11.122966, -17.239277],
    [-28832.0, 20976.0, 1246.0, -17.419025, -13.899675, -23.136898],
    [-27144.0, 16766.0, -952.0, -22.814985, -20.076349, -33.424778],
    [-25726.0, -9456.0, -836.0, -31.062357, -25.452982, -36.934464],
    [-23457.0, -6729.0, 8890.0, -36.388248, -34.632843, -41.479675],
    [-13342.0, 5495.0, 15403.0, -38.879993, -39.638306, -46.69796],
    [-13950.0, 4022.0, 11844.0, -36.205753, -42.41032, -48.735119],
    [-17379.0, 1175.0, 3612.0, -29.3762, -40.888321, -43.87096],
    [-17644.0, 5237.0, 906.0, -25.636518, -35.147442, -36.420631],
    [-13387.0, 14061.0, 1121.0, -20.844109, -30.878424, -35.420471],
    [-5536.0, 17643.0, 2526.0, -17.503853, -25.282803, -37.302792],
    [-441.0, 18073.0, 5692.0, -12.796161, -22.123541, -37.482185],
    [-1052.0, 13925.0, 5458.0, -10.660426, -17.78968, -33.799526],
    [1416.0, 3755.0, 3915.0, -8.636809, -15.191227, -28.779388],
    [4216.0, 704.0, -45.0, -7.520261, -12.325178, -21.302486],
    [3867.0, 7068.0, 706.0, -5.716715, -10.736494, -18.8957],
    [3917.0, 4603.0, -865.0, -4.368924, -7.545374, -17.54945],
    [4891.0, 163.0, -3398.0, -2.847019, -5.442245, -15.319879],
    [2282.0, -2054.0, -2474.0, -2.491301, -3.002312, -10.408621],
    [2751.0, -7751.0, 843.0, -4.044754, -2.411417, -7.670542],
    [-898.0, -12807.0, 2671.0, -5.888142, -3.370853, -4.061652]
]

predictions1 = predict_new_data(new_data1, model, scaler, label_encoder)
print("first batch")
print(predictions1)

predictions2 = predict_new_data(new_data2, model, scaler, label_encoder)
print("second batch")
print(predictions2)

first batch
['Neutral' 'Neutral' 'Neutral' 'Neutral' 'Neutral' 'Crouch' 'Crouch'
 'Crouch' 'Crouch' 'Crouch' 'Crouch' 'Crouch' 'Crouch' 'Crouch' 'Crouch'
 'Crouch' 'Crouch' 'Crouch' 'Crouch' 'Crouch' 'Crouch' 'Crouch' 'Neutral'
 'Neutral' 'Neutral' 'Neutral' 'Neutral']
second batch
['Neutral' 'Neutral' 'Neutral' 'Neutral' 'Neutral' 'Neutral' 'Shoot'
 'Shoot' 'Shoot' 'Shoot' 'Shoot' 'Shoot' 'Shoot' 'Shoot' 'Shoot' 'Shoot'
 'Shoot' 'Shoot' 'Neutral' 'Neutral' 'Neutral' 'Neutral' 'Neutral'
 'Neutral' 'Neutral']


In [74]:
# Laden und Verfeinern des Modells
# Modell neu instanziieren und Zustand laden
model = LSTMClassifier(input_size, hidden_size, output_size, num_layers)
model.load_state_dict(torch.load('gesture_model.pth'))

# Optional: Modell weiter trainieren (feinabstimmen)
model.train()
for epoch in range(num_epochs):
    for X_batch, y_batch in train_loader:
        X_batch = X_batch.unsqueeze(1)  # Batch-Dimension hinzufügen für LSTM
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Fine-tuning Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Speichern Sie das verfeinerte Modell
torch.save(model.state_dict(), 'refined_gesture_model.pth')

Fine-tuning Epoch [1/500], Loss: 0.0000
Fine-tuning Epoch [2/500], Loss: 0.0000
Fine-tuning Epoch [3/500], Loss: 0.0000
Fine-tuning Epoch [4/500], Loss: 0.0000
Fine-tuning Epoch [5/500], Loss: 0.0000
Fine-tuning Epoch [6/500], Loss: 0.0000
Fine-tuning Epoch [7/500], Loss: 0.0000
Fine-tuning Epoch [8/500], Loss: 0.0000
Fine-tuning Epoch [9/500], Loss: 0.0000
Fine-tuning Epoch [10/500], Loss: 0.0000
Fine-tuning Epoch [11/500], Loss: 0.0000
Fine-tuning Epoch [12/500], Loss: 0.0000
Fine-tuning Epoch [13/500], Loss: 0.0000
Fine-tuning Epoch [14/500], Loss: 0.0001
Fine-tuning Epoch [15/500], Loss: 0.0000
Fine-tuning Epoch [16/500], Loss: 0.0000
Fine-tuning Epoch [17/500], Loss: 0.0000
Fine-tuning Epoch [18/500], Loss: 0.0000
Fine-tuning Epoch [19/500], Loss: 0.0000
Fine-tuning Epoch [20/500], Loss: 0.0000
Fine-tuning Epoch [21/500], Loss: 0.0000
Fine-tuning Epoch [22/500], Loss: 0.0000
Fine-tuning Epoch [23/500], Loss: 0.0000
Fine-tuning Epoch [24/500], Loss: 0.0000
Fine-tuning Epoch [25/500

In [75]:
# Konvertieren des PyTorch-Modells in ONNX
dummy_input = torch.randn(1, 1, input_size)
torch.onnx.export(
    model,
    dummy_input,
    "IMU_lstm_model.onnx",
    input_names=['input'],
    output_names=['output']
)

/usr/local/lib/python3.10/dist-packages/torch/onnx/symbolic_opset9.py:4661: UserWarning: Exporting a model to ONNX with a batch_size other than 1, with a variable length with LSTM can cause an error when running the ONNX model with a different batch size. Make sure to save the model with a batch size of 1, or define the initial states (h0/c0) as inputs of the model. 
  warnings.warn(
